#Final project - Deep Learning COLLMAN 2020

In this project you are requested to develop an algorithm that predicts the winner of a given Tennis match.

In order to do that, you will be working on two networks:
1. Embedding network - trained to predict a single outcome, given two players, in order to learn how to represent a player using a vector.
2. Prediction network - given two tennis player, this network predicts who is about to win. Each player is represented by a vector that is learned by the embedding network.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import random
from torch.nn.functional import one_hot
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Dataset**

The dataset is composed of a collection of Tennis matches and their outcomes, provided in multiple files. You should download the data from https://drive.google.com/open?id=1ie6WSl8qkknpSGAFt32Gj7Tf0JAhTLu0 and place the folder somewhere in your Google drive. We provide you the following function *read_data_for_embedding_network*, which reads the files and create a few dictionaries that will help you handle the data later, and a list of instances for training the embedding network. Here is a detailed description of the output of this function:

1. player_names - this is a dictionary that maps between a player id and a player name. This player id is unique, and it is given by the dataset owners.
2. player_id2index - this is a dictionary that maps between a player id and an index (starting at 0). This index is unique, and it is created by us. It will help us represent the players with a one-hot vector later.
3. players_n - an integer representing the number of players we have in the data (this will be the length of a one-hot vector representing a player).
4. embedding_data - the dataset to train the embedding network. This dataset is a simple list of lists. Each internal list represents an individual training instance, coomposed of three components: the winner index, the loser index and a binary represnting who scored more aces (a serve that was not responded) during the match.

In [ ]:
def read_data_for_embedding_network(folder_path):
  player_names = {}
  player_id2index = {}
  players_n = 0
  embedding_data = []
  pathlist = Path(folder_path).glob('**/*.csv')
  for path in pathlist:
     path_in_str = str(path)
     print(path_in_str)
     raw = pd.read_csv(path_in_str, low_memory=False)
     for i, match in raw.iterrows():
      if match['winner_id'] not in player_names:
        player_names[match['winner_id']] = match['winner_name']
        player_id2index[match['winner_id']] = players_n
        players_n += 1
      if match['loser_id'] not in player_names:
        player_names[match['loser_id']] = match['loser_name']
        player_id2index[match['loser_id']] = players_n
        players_n += 1
      embedding_data.append([player_id2index[match['winner_id']], 
                             player_id2index[match['loser_id']], 
                            int(match['w_ace'] > match['l_ace'])])
  return player_names, player_id2index, players_n, embedding_data

#TODO - put your own path for the dataset folder
player_names, player_id2index, players_n, embedding_data = read_data_for_embedding_network('/content/drive/My Drive/atp-matches-dataset') 


/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2000.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2001.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2002.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2003.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2004.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2005.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2006.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2007.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2008.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2009.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2010.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/a

In [ ]:
print(players_n)
print(len(player_names))
print(player_names)
print(len(player_id2index))
print(player_id2index)

2053
2053
{102179: 'Antony Dupuis', 102776: 'Andrew Ilie', 103602: 'Fernando Gonzalez', 102821: 'Cecil Mamiit', 103387: 'Paradorn Srichaphan', 102205: 'Sebastien Lareau', 101733: 'Jan Siemerink', 102925: 'Justin Gimelstob', 101727: 'Jason Stoltenberg', 101826: 'Alex Lopez Moron', 103181: 'Jiri Vanek', 101885: 'Wayne Arthurs', 101675: 'Martin Rodriguez', 102455: 'Lorenzo Manta', 102834: 'Paul Goldstein', 101086: 'Ronald Agenor', 103454: 'Nicolas Massu', 102133: 'Laurence Tieleman', 102466: 'Jan Kroslak', 102966: 'Fredrik Jonsson', 103188: 'Michael Russell', 103204: 'Hugo Armando', 101150: 'Gianluca Pozzi', 102880: 'Alexander Popp', 103009: 'Razvan Sabau', 102987: 'Andre Sa', 103082: 'Markus Hantschk', 102720: 'Tomas Zib', 103598: 'Xavier Malisse', 102233: 'Leander Paes', 102887: 'Ramon Delgado', 102998: 'Jan Michael Gambill', 101736: 'Andre Agassi', 103420: 'Michal Tabara', 101320: 'Magnus Gustafsson', 101543: 'Jeff Tarango', 101723: 'Magnus Larsson', 102104: 'Christian Ruud', 102872: '

**Embedding network**

Here you are requested to code the embedding network. An embedding network in general, is created for learning how to represent an input provided as a one-hot vector. It usually "projects" the one-hot vectors on another vector space, which has less dimensions that capture some meaningful information about the inputs. In this case, we will feed the network with the instances we got from the previous function: one hot vector to represent the winner, and ont hot vector to represent the loser. The network will try to predict who scored more aces. You should implement the network, based on the architecture depicted in the followin diagram. **Note - use ReLU activation, only after Linear 1.**

![alt text](https://docs.google.com/drawings/d/e/2PACX-1vRp5PeqjF_W-gQmpjcY32hx4n5kWKsySdvLr3avz-bc0kodQbSlvNFuTBX6lcLzS6eS1nwjW0IdVCTY/pub?w=960&h=720)

In [ ]:
# TODO - complete the following network based on the diagram that's mentioned above.

class EmbeddingsNet(torch.nn.Module):

    def __init__(self, players_n, hidden_size):
        '''
        players_n - the size of the one hot vectors representing a single player
        hidden_size - the output size of Linear of winner, Linear of loser and Linear 1 from the diagram
        Note - Linear 2's output size should be 2
        '''
        super(EmbeddingsNet, self).__init__()
        # TODO
        self.embeddings = nn.Linear(players_n, hidden_size)
        self.embeddings2 = nn.Linear(players_n, hidden_size)
        self.linear1 = nn.Linear(hidden_size*2, hidden_size)
        self.linear2 = nn.Linear(hidden_size, 2)

    def forward(self, player_1_batch, player_2_batch):
        '''
        player_1_batch - Bx(number of players) tensor - a batch of winner one hot vectors
        player_2_batch - Bx(number of players) tensor - a batch of loser one hot vectors
        '''
        # TODO
        
        embeds = self.embeddings(player_1_batch)
        embeds2 = self.embeddings2(player_2_batch)
        concatenated = torch.cat((embeds, embeds2), dim=1)
        out = F.relu(self.linear1(concatenated))
        out = self.linear2(out)
        

        return out

**Training the embedding network**

You are requested to write a function that trains the embedding network.

In [ ]:
# Intializing the embedding network:

embedding_net = EmbeddingsNet(players_n, 200).cuda()
embedding_net.eval()

EmbeddingsNet(
  (embeddings): Linear(in_features=2053, out_features=200, bias=True)
  (embeddings2): Linear(in_features=2053, out_features=200, bias=True)
  (linear1): Linear(in_features=400, out_features=200, bias=True)
  (linear2): Linear(in_features=200, out_features=2, bias=True)
)

In [ ]:

optimizer = optim.Adam(embedding_net.parameters(), lr=0.001)


In [ ]:
# TODO - you should implement the following function. You should define 
# all the relevant tools required to train the network (e.g., optimizer, loss function) 
# inside this function. Please use Adam optimizer with default parameters.
# We suggest that you print the loss function every once in a while so that you 
# can watch the network improves. 
def batch_data(batch):
    x,y = batch.shape
    winner_batch = np.zeros((x,), dtype=np.float64) 
    loser_batch =np.zeros((x,), dtype=np.float64) 
    aces_batch = np.zeros((x,), dtype=np.float64) 
    for i in range(x):
        winner_batch[i] = batch[i][0]
        loser_batch[i] = batch[i][1]
        aces_batch[i] = batch[i][2]
    
    return winner_batch,loser_batch,aces_batch
def toTensorA(winner_batch,loser_batch):
    x= winner_batch.size
    
    for i in range(x):
        winner_batch[i]=torch.tensor([[winner_batch[i]]], dtype=torch.float32  ).cuda()
        loser_batch[i]=torch.tensor([[loser_batch[i]]], dtype=torch.float32  ).cuda()
    winner_batch=torch.tensor(winner_batch,dtype=torch.float32  ).cuda()
    loser_batch=torch.tensor(loser_batch,dtype=torch.float32  ).cuda()
    return winner_batch,loser_batch
def aces(aces_batch):
    x= aces_batch.size
    aces_batch_new = torch.zeros([x,1], dtype=torch.float32  ).cuda()
    aces_batch_new2 = torch.zeros([x,1], dtype=torch.float32  ).cuda()
    for i in range(x):
        if(aces_batch[i] == 0):
            aces_batch_new[i] = 1
            aces_batch_new2[i] = 0
        else:
            aces_batch_new[i] = 0
            aces_batch_new2[i] = 1 
    concatenated2 = torch.cat((aces_batch_new, aces_batch_new2), dim=0).cuda()
    return concatenated2
def get_input_layer(word_idx):
    a = torch.tensor(len(word_idx),2053).cuda()
    x = torch.zeros(2053).float().cuda()
    for i in range(len(word_idx)):
        x[word_idx[i]] = 1.0
        a.append(x[word_idx[i]])
    return list
def train_embeddings_network(model, data, epochs, batch_size):
    '''
      model - the embedding_net object
      data - the dataset (i.e., embedding_data from above)
      epochs - number of epochs to train the network
      batch_size - the batch size
    '''
    N = len(data)
    for epoch in range(epochs):
        total_loss=0
        indexes = np.random.permutation(N)
        new_data = np.array(data)
        shuffled_data = new_data[indexes]
        batches = [shuffled_data[b:b+batch_size]for b in range(0, N, batch_size)]
        
        for i , batch in enumerate(batches, 0):
            
            winner_batch , loser_batch , more_aces = batch_data(batch)
            winner_batch_new,loser_batch_new=toTensorA(winner_batch,loser_batch)
            
            winner_idxs=F.one_hot(torch.tensor(winner_batch).to(torch.int64), num_classes=2053).cuda()
            winner_idxs=winner_idxs.float()
            
            
            loser_idxs=F.one_hot(torch.tensor(loser_batch).to(torch.int64), num_classes=2053).cuda()
            loser_idxs=loser_idxs.float()
            
            model.zero_grad()

            outputs = model(winner_idxs , loser_idxs)
            aces_batch_new=torch.tensor(more_aces,dtype=torch.long).cuda()
            loss = F.cross_entropy(outputs,aces_batch_new)
            loss.backward() #calculate the gradients
            #model.model_params['W1_winner']-(0.001*model.model_params['W1_winner'].grad.data)
            optimizer.step()#updating the weights
            total_loss+=loss.item()
        print(f'Epoch {epoch}, loss={total_loss/len(batches)}')

 


In [ ]:
# call the training procedure you implemented:

train_embeddings_network(embedding_net, embedding_data, 40, 8)

Epoch 0, loss=0.5635464106075109
Epoch 1, loss=0.5250478385299505
Epoch 2, loss=0.5127879498826139
Epoch 3, loss=0.5054453008113793
Epoch 4, loss=0.4993298167663084
Epoch 5, loss=0.49417637788222535
Epoch 6, loss=0.48884668809772386
Epoch 7, loss=0.4834917451260475
Epoch 8, loss=0.4770186937045323
Epoch 9, loss=0.47113675722141296
Epoch 10, loss=0.46575917695362984
Epoch 11, loss=0.4610215332255939
Epoch 12, loss=0.4567109247296223
Epoch 13, loss=0.45211601345078645
Epoch 14, loss=0.44775821646509567
Epoch 15, loss=0.4449532014927642
Epoch 16, loss=0.4408414074124141
Epoch 17, loss=0.43633535640009824
Epoch 18, loss=0.43243668530842977
Epoch 19, loss=0.42828747546785034
Epoch 20, loss=0.42507674850657845
Epoch 21, loss=0.4223369113497437
Epoch 22, loss=0.4188707887594778
Epoch 23, loss=0.4162032854049488
Epoch 24, loss=0.4131788997113082
Epoch 25, loss=0.4117028089521609
Epoch 26, loss=0.40697754408797027
Epoch 27, loss=0.4046158808613692
Epoch 28, loss=0.40331543157703137
Epoch 29, lo

**Extracting embeddings**

In this section, we will ask you to extract the player representation from the embedding network. In fact, we don't realy care about the quality of prediction of the embedding network. We care more about the vector representation it learned for every player. The representation is in fact the weight matrix of the two first linear layers that you created for each player of a given player pair. Each such weight matrix is of size (hidden_size X number_of_players). You can get that matrix of each of the two Linear layers, by calling their *parameters* function. This function returns a Python generator, and the first element is your matrix. Note that since you have two such layers, you will have two matrices; you are requested to add the two matrices together and divide by 2, and then you get a new matrix which will represent the players. This matrix, say M, will be of shape (hidden_size X number_of_players), so **M[:, 3]** for example, is a hidden_size vector representing player of index 3.

In [ ]:
# TODO: preparing the embedding matrix, let's call it player_embedding

player_embedding = (embedding_net.embeddings.weight.data+embedding_net.embeddings2.weight.data)/2

print(player_embedding)
print(player_embedding.shape)


tensor([[ 0.2590,  0.5316, -0.4917,  ...,  0.2403,  0.1848,  0.6200],
        [ 0.4401, -0.6148, -0.6617,  ...,  0.2966, -0.1155, -0.5324],
        [-0.0531,  0.2890, -0.4194,  ..., -0.2272,  0.6483,  0.5971],
        ...,
        [-0.9189,  0.0913, -0.5275,  ..., -0.6876, -0.4237, -0.6049],
        [ 0.1470, -0.9408, -0.0343,  ...,  0.2217, -0.6655, -0.6280],
        [-0.0146, -0.3992,  0.0930,  ...,  0.2534, -0.4056, -0.5169]],
       device='cuda:0')
torch.Size([200, 2053])


**Prediction network**

Now, we are ready to implement a simple network that predicts who wins a match, given two players and some additional match-leavel features. Please implement PredictionNetwork based on the following diagram. Each player is provided with it's embedding from the player_embedding matrix you calculated above. **Note - use ReLU after every Linear layer (except the last one of course).**


![alt text](https://docs.google.com/drawings/d/e/2PACX-1vTCAlmaool-UQ9be8wSdjjgOq0VaCiYsXwsHw0HDxZRXTkp6vBm08Ma8sQha-cKDjjeZhfzY2qT8if8/pub?w=960&h=720)

In [ ]:
class PredictionNetwork(torch.nn.Module):
  def __init__(self, players_embedding_size, match_feature_size, hidden_size1, hidden_size2, hidden_size3):
    '''
      players_embedding_size - the length of the vector representing each player 
      (basically, the hidden_size of the embedding network)
      match_feature_size - the number of match-level features we use
      hidden_size1 - the output size of all the three first layers (Linear player 1, 
      Linear player 2, and Linear features)
      hidden_size2 - the output size of Linear 1
      hidden_size3 - the output size of Linear 2
    '''
    super(PredictionNetwork, self).__init__()
    # TODO
    self.embeddings = nn.Linear(players_embedding_size, hidden_size1)
    self.embeddings2 = nn.Linear(players_embedding_size, hidden_size1)
    self.embeddings3 = nn.Linear(match_feature_size, hidden_size1)
    self.linear1 = nn.Linear(hidden_size1*3, hidden_size2)
    self.linear2 = nn.Linear(hidden_size2, hidden_size3)
    self.linear3 = nn.Linear(hidden_size3, 2)

  def forward(self, player1_batch, player2_batch, match_features_batch):
    '''
      player1_batch - Bx(players_embedding_size) tensor - a batch of player 1 embedding vectors
      player2_batch - Bx(players_embedding_size) tensor - a batch of player 2 embedding vectors
      match_features_batch - Bx(match_feature_size) tensor - a batch of match-level features
    '''
    # TODO
    embed=self.embeddings(player1_batch)
    embed2=self.embeddings2(player2_batch)
    embed3=self.embeddings3(match_features_batch)
    concatenated = torch.cat((embed, embed2,embed3), dim=1).cuda()
    out1 = F.relu(self.linear1(concatenated))
    out2 = F.relu(self.linear2(out1))
    out3 = self.linear3(out2)
    return out3

The following function, **prepare_match_dataset**, prepares the dataset of instances required to train your prediction network. We randomize the position of the winner and loser in each instance, to avoid converging to position based prediction. As you can see, we only use only a single match-level feature for now: draw_size.

In [ ]:
def prepare_match_dataset(folder_path, player_embedding):
  X = []
  y = []
  surface_dictionary = {}
  pathlist = Path(folder_path).glob('**/*.csv')
  for path in pathlist:
    path_in_str = str(path)
    print(path_in_str)
    raw = pd.read_csv(path_in_str, low_memory=False)
    for i, match in raw.iterrows():
      instance = []
      features = []
      features.append(match['draw_size'])
      winner_embedding = player_embedding[:, player_id2index[match['winner_id']]].cpu().detach().numpy()
      loser_embedding = player_embedding[:, player_id2index[match['loser_id']]].cpu().detach().numpy()
      target = -1
      if random.random() < 0.5:
         instance.append(winner_embedding)
         instance.append(loser_embedding)
         target = 0
      else:
         instance.append(loser_embedding)
         instance.append(winner_embedding)
         target = 1
      instance.append(features)
      X.append(instance)
      y.append(target)

  return X, y


#TODO - put your own path for the dataset folder
X, y = prepare_match_dataset('/content/drive/My Drive/atp-matches-dataset', player_embedding) 


/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2000.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2001.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2002.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2003.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2004.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2005.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2006.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2007.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2008.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2009.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/atp_matches_2010.csv
/content/drive/My Drive/atp-matches-dataset/atp-matches-dataset/a

**Training the prediction network**

Now, you should write code to train the prediction network using the X,y dataset.

In [ ]:
# TODO - split the data into train and test. Please allocate 25% of the data for test, using random_state=42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [ ]:
# TODO - you should implement the following train_match_network function. You should define 
# all the relevant tools required to train the network (e.g., optimizer, loss function) 
# inside this function. Please use Adam optimizer with default parameters.
# You should print and plot the loss function for the train data every once in a while, 
# as well as accuracy for the test data (every epoch) so that you 
# can watch the network improves.
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()
def batch_data2(batch):
    x,y = batch.shape
    winner_batch = []
    loser_batch =[]
    features_batch = []
    for i in range(x):
        winner_batch.append(batch[i][0])
        loser_batch.append(batch[i][1])
        features_batch.append(batch[i][2])
    
    return winner_batch,loser_batch,features_batch
def batch_data_test(batch):
    x=len(batch)

    winner_batch = []
    loser_batch =[]
    features_batch = []
    for i in range(x):
        winner_batch.append(batch[i][0])
        loser_batch.append(batch[i][1])
        features_batch.append(batch[i][2])
    
    return winner_batch,loser_batch,features_batch   
def toTensor(winner_batch,loser_batch,features_batch):
    x= len(winner_batch)
    winner_batch_1 = torch.zeros([x,len(winner_batch[0])], dtype=torch.float).cuda() 
    loser_batch_1 =torch.zeros([x,len(loser_batch[0])], dtype=torch.float).cuda() 
    features_batch_1 = torch.zeros([x,len(features_batch[0])], dtype=torch.float).cuda() 

    for i in range(x):
        winner_batch_1[i]=torch.tensor([[winner_batch[i]]], dtype=torch.float  ).cuda()
        loser_batch_1[i]=torch.tensor([[loser_batch[i]]], dtype=torch.float  ).cuda()
        features_batch_1[i]=torch.tensor([[features_batch[i]]], dtype=torch.float  ).cuda()
    return winner_batch_1,loser_batch_1,features_batch_1
def label_batch(batch_y):
    x= len(batch_y)
    batch_y_new = torch.zeros([x], dtype=torch.long  ).cuda()
    for i in range(x):
        batch_y_new[i] = torch.tensor([[batch_y[i]]], dtype=torch.long  ).cuda()
    return  batch_y_new
def train_match_network(model, X_train, X_test, y_train, y_test, epochs, batch_size):
    '''
      X_train - the training data
      X_test - the testing data
      y_train - the target of the training data
      y_test - the target of the testing data
      epochs - number of epochs
      batch_size - the batch size
    '''
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    N = len(y_train)
    N2 = len(y_test)
    for epoch in range(epochs):
        total_loss=0
        total_test_correct = 0
        total_test_loss=0
        indexes = np.random.permutation(N)
        new_data_X = np.array(X_train)
        new_data_y = np.array(y_train)
        shuffled_X = new_data_X[indexes]
        shuffled_y = new_data_y[indexes]
        batches = [(shuffled_X[b:b+batch_size], shuffled_y[b:b+batch_size]) for b in range(0, N, batch_size)]
        for i , batch in enumerate(batches, 0):
            batch_x , batch_y = batch
            player1_batch, player2_batch, match_features_batch = batch_data2(batch_x)
            player1_batch_new,player2_batch_new,match_features_batch_new=toTensor(player1_batch,player2_batch,match_features_batch)
            
            model.zero_grad()

            outputs = model(player1_batch_new , player2_batch_new,match_features_batch_new)
            labels=label_batch(batch_y)
            loss = F.cross_entropy(outputs,labels)
            loss.backward() #calculate the gradients
            optimizer.step()#updating the weights
            total_loss+=loss.item()
        print(f'Epoch {epoch}, Train_loss={total_loss/len(batches)}')
        
        with torch.no_grad():
            
            test_batches = [(X_test[b:b+batch_size], y_test[b:b+batch_size]) for b in range(0, N2, batch_size)]
            for i, batch_test in enumerate(test_batches, 0):

                batch_x_test , batch_y_test = batch_test
                player1_batch_t, player2_batch_t, match_features_batch_t =  batch_data_test(batch_x_test)
                player1_batch_newt,player2_batch_newt,match_features_batch_newt=toTensor(player1_batch_t,player2_batch_t,match_features_batch_t)


                preds = model(player1_batch_newt,player2_batch_newt,match_features_batch_newt) #pass batch
                labels_test=label_batch(batch_y_test)
                loss_test = F.cross_entropy(preds,labels_test) #calculating the loss

                total_test_loss+=loss_test.item()
                total_test_correct+=get_num_correct(preds,labels_test)
                
                
        
        print("epoch:",epoch,'Accuracy of the network on the test data: %d %%' % (100 * total_test_correct / len(y_test)),"total_test_loss:",total_test_loss/len(test_batches))



        

In [ ]:
# TODO - train your network
tennisMatchModel = PredictionNetwork(200, 1, 1024, 700, 200).cuda()
tennisMatchModel.eval()


PredictionNetwork(
  (embeddings): Linear(in_features=200, out_features=1024, bias=True)
  (embeddings2): Linear(in_features=200, out_features=1024, bias=True)
  (embeddings3): Linear(in_features=1, out_features=1024, bias=True)
  (linear1): Linear(in_features=3072, out_features=700, bias=True)
  (linear2): Linear(in_features=700, out_features=200, bias=True)
  (linear3): Linear(in_features=200, out_features=2, bias=True)
)

In [ ]:
train_match_network(tennisMatchModel, X_train, X_test, y_train, y_test, 40, 2)

Epoch 0, Train_loss=0.7183660492006312
epoch: 0 Accuracy of the network on the test data: 48 % total_test_loss: 0.6933120397275959
Epoch 1, Train_loss=0.6956626022534048
epoch: 1 Accuracy of the network on the test data: 52 % total_test_loss: 0.691457907740705
Epoch 2, Train_loss=0.6966661627578791
epoch: 2 Accuracy of the network on the test data: 51 % total_test_loss: 0.6944964904938671
Epoch 3, Train_loss=0.7065391327244995
epoch: 3 Accuracy of the network on the test data: 52 % total_test_loss: 0.6956173870697785
Epoch 4, Train_loss=0.6937775312596934
epoch: 4 Accuracy of the network on the test data: 52 % total_test_loss: 0.6889319612838067
Epoch 5, Train_loss=0.6936074268454547
epoch: 5 Accuracy of the network on the test data: 49 % total_test_loss: 0.6901566360951451
Epoch 6, Train_loss=0.6943253479685416
epoch: 6 Accuracy of the network on the test data: 52 % total_test_loss: 0.6885988657807572
Epoch 7, Train_loss=0.6913829329886002
epoch: 7 Accuracy of the network on the test 